Copyright (c) 2022 José Ángel de Bustos Pérez

May be copied or modified under the terms of the GNU General Public License v3.0.  See https://www.gnu.org/licenses/gpl-3.0.html

# Training Xception

This notebook trains Xception models and save the best and the last model.

The h5 model is not included due to it exceeds GitHub storage limits.


## Check environment

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)    

print("Tensorflow version: %s\n" % tf.__version__)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## Hyperparameters and data preparation

In [ ]:
import numpy as np
import string
import os

imgs_base = '/opt/app-root/src/rhods/dataset'
imgs_path_train = os.path.join(imgs_base,'train')
imgs_path_validation = os.path.join(imgs_base,'validation')
imgs_path_test = os.path.join(imgs_base,'test')

# Hyperparams
IMAGE_SIZE = 200
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE, IMAGE_SIZE
EPOCHS = 4
BATCH_SIZE = 16

DROPOUT = 0.5

labels = ['violence', 'nonviolence']
labels_dict = {}
labels_dict['violence'] = 0
labels_dict['nonviolence'] = 1

# get files in directory
def get_files_dir(directory):
    return os.listdir(directory)

files_dir = get_files_dir(os.path.join(imgs_path_train, 'violence'))
print("Training images (violence): %d" % len(files_dir))
files_dir = get_files_dir(os.path.join(imgs_path_train, 'nonviolence'))
print("Training images (nonviolence): %d" % len(files_dir))

files_dir = get_files_dir(os.path.join(imgs_path_validation, 'violence'))
print("Validation images (violence): %d" % len(files_dir))
files_dir = get_files_dir(os.path.join(imgs_path_validation, 'nonviolence'))
print("Validation images (nonviolence): %d" % len(files_dir))

Load training and validation data:

In [ ]:
import glob
import cv2
from glob import glob

def load_set(dirname, labels_dict, verbose=True):
    """Esta función carga los datos de training en imágenes.
    
    Como las imágenes tienen tamaños distintas, utilizamos la librería opencv
    para hacer un resize y adaptarlas todas a tamaño IMG_SIZE x IMG_SIZE.
    
    Args:
        dirname: directorio completo del que leer los datos
        verbose: si es True, muestra información de las imágenes cargadas
     
    Returns:
        X, y: X es un array con todas las imágenes cargadas con tamaño
                IMG_SIZE x IMG_SIZE
              y es un array con las labels de correspondientes a cada imagen
    """
    labels = glob(os.path.join(dirname, '*'), recursive = False)
    X_ = []
    y_ = []
    
    for label in labels:
        label = label.split('/')[-1]
        dir_path = os.path.join(dirname, label)
        files_path = os.listdir(dir_path)
        images = [file for file in files_path if file.endswith("png")]
        if verbose:
          print("Reading {} images found in {}".format(len(images), files_path))
        for image_name in images:
            image = cv2.imread(os.path.join(dir_path,image_name))
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            #X_.append(cv2.resize(image,(IMAGE_WIDTH, IMAGE_HEIGHT)))
            X_.append(image)
            y_.append(labels_dict[label])
    return (np.array(X_).astype('float32'))/255, np.array(y_)

# load data
print("Loading training data ...")
X_train, y_train = load_set(imgs_path_train, labels_dict, verbose=False)
print("Loading validation data ...")
X_validation, y_validation = load_set(imgs_path_validation, labels_dict, verbose=False)

# define input_shape
if tf.keras.backend.image_data_format() == 'channels_first':
    input_shape = (3, IMAGE_WIDTH, IMAGE_HEIGHT)
else:
    input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)

## Model creation

In [ ]:
import time
import tensorflow as tf
import tensorflow.keras as K

# function to get training time (human readable)
def display_time(seconds, granularity=2):
    result = []

    intervals = (
        ('dias', 86400),    # 60 * 60 * 24
        ('horas', 3600),    # 60 * 60
        ('minutos', 60),
        ('segundos', 1),
    )
    
    for name, count in intervals:
        value = seconds // count
        if value:
            seconds -= value * count
            if value == 1:
                name = name.rstrip('s')
            result.append("{} {}".format(value, name))
    return ', '.join(result[:granularity])

# TODO MOVE NEURAL NETWORK CODE TO A CLASS

# load the Keras xception
xception_model = tf.keras.applications.Xception(
    include_top = False,
    weights = 'imagenet',
#    input_tensor = K.Input(shape=input_shape),
    input_shape = input_shape,
)

for layer in xception_model.layers:
    layer.trainable = False

model = K.Sequential()
model.add(xception_model)
model.add(K.layers.Flatten())
model.add(K.layers.Dense(512, activation="relu"))
model.add(K.layers.Dropout(DROPOUT))
model.add(K.layers.Dense(512, activation="relu"))
model.add(K.layers.Dropout(DROPOUT))
model.add(K.layers.Dense(16, activation="relu"))
model.add(K.layers.Dropout(DROPOUT))
model.add(K.layers.Dense(16, activation="relu"))
model.add(K.layers.Dense(1, activation='sigmoid'))

VERSION="alt3"

model_checkpoint_filename_acc = "trained-xception-b" + str(BATCH_SIZE) + "-e" + str(EPOCHS) + "-16frames-acc-dropout-" + str(DROPOUT) + "-" + VERSION + ".h5"
check_point = K.callbacks.ModelCheckpoint(filepath=model_checkpoint_filename_acc,
                                          monitor="val_accuracy",
                                          mode="max",
                                          save_best_only=True,
                                         )

#early_stopping = K.callbacks.EarlyStopping(monitor='val_accuracy', patience=16, verbose=0, mode='max')

# loss:
#  binary_crossentropy for binary problems
#  categorical_crossentropy for more classes

lr = 1e-6
model.compile(loss='binary_crossentropy',
                 optimizer=K.optimizers.RMSprop(learning_rate=lr),
                  metrics=['accuracy'])

model.summary()

## Training the model

Callback to see the model performance. **PlotLossesKeras** from **livelossplot** does not work, it seem a problem with Tensorflow version (2.4.1) and Keras version (2.9.0).

In [ ]:
import numpy as np 
from tensorflow import keras
from matplotlib import pyplot as plt
from IPython.display import clear_output

class PlotLearning(keras.callbacks.Callback):
    """
    Callback to plot the learning curves of the model during training.
    """
    def on_train_begin(self, logs={}):
        self.metrics = {}
        for metric in logs:
            self.metrics[metric] = []
            

    def on_epoch_end(self, epoch, logs={}):
        # Storing metrics
        for metric in logs:
            if metric in self.metrics:
                self.metrics[metric].append(logs.get(metric))
            else:
                self.metrics[metric] = [logs.get(metric)]
        
        # Plotting
        metrics = [x for x in logs if 'val' not in x]
        
        f, axs = plt.subplots(1, len(metrics), figsize=(15,5))
        clear_output(wait=True)

        for i, metric in enumerate(metrics):
            axs[i].plot(range(1, epoch + 2), 
                        self.metrics[metric], 
                        label=metric)
            if logs['val_' + metric]:
                axs[i].plot(range(1, epoch + 2), 
                            self.metrics['val_' + metric], 
                            label='val_' + metric)
                
            axs[i].legend()
            axs[i].grid()

        plt.tight_layout()
        plt.show()

In [ ]:
time.ctime()

In [ ]:
# time to start training
start_time = time.time()

history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1,
                    validation_data=(X_validation, y_validation),
                    shuffle=True,
                    callbacks=[PlotLearning(),
                               check_point])

# time at the end of training
end_time = time.time()

model_last_filename = "trained-xception-b" + str(BATCH_SIZE) + "-e" + str(EPOCHS) + "-16frames-last-dropout-" + str(DROPOUT) + "-" + VERSION + ".h5"

model.summary()
model.save(model_last_filename)

# time to train
msg = display_time(end_time - start_time)
print("Training time: %s" % msg)

In [ ]:
accuracy = history.history['accuracy']
max(accuracy)

In [ ]:
loss = history.history['loss']
min(loss)

In [ ]:
val_accuracy = history.history['val_accuracy']
max(val_accuracy)

In [ ]:
val_loss = history.history['val_loss']
min(val_loss)

## Testing the model

In [ ]:
# load data
print("Loading test data ...")
X_test, y_test = load_set(imgs_path_test, labels_dict, verbose=False)

prediction = model.predict(X_test[:1])

pred_value = prediction[0,0]
true_value = [k for k, v in labels_dict.items() if v == y_test[:1][0]]

if pred_value >= 0.5:
    msg = 'Algoritm has predicted non-violence with a probability of %f. The true label is non-violence.' % pred_value
else:
    msg = 'Algoritm has predicte violence with a probability of %f. The true label is non-violence.' % pred_value
    
print(msg)